First import relevant libraries. Notably ones to deal with time and plotting time data on the $x$ axis.

In [88]:
import numpy as np
import pandas as pd
import time
import datetime as dt
import matplotlib.dates as md
import os
import matplotlib.pyplot as plt
print(os.listdir("../input"))

Selecting the bitcoin prices from coinbase.

In [89]:
coinbase=pd.read_csv('../input/coinbaseUSD_1-min_data_2014-12-01_to_2018-01-08.csv')
coinbase.head(5)

In [90]:
print(coinbase.shape)

I only want to look at data from the year 2017. Find time stamps for 1/1/17 and 1/1/18.

In [91]:
s = "01/01/2017"
sts=time.mktime(dt.datetime.strptime(s, "%d/%m/%Y").timetuple())
print(sts)
e = "01/01/2018"
ets=time.mktime(dt.datetime.strptime(e, "%d/%m/%Y").timetuple())
print(ets)

Select rows with timestamps between the above two.

In [92]:
index=(coinbase['Timestamp']>=sts)&(coinbase['Timestamp']<ets)
coinbase17=coinbase[index]
coinbase17.shape

Create a function to convert a timestamp to readable date format. Apply it and use it as index

In [93]:
def totime(x):
    return dt.datetime.fromtimestamp(x).strftime('%m/%d %H:%M:%S')
coinbase17.index = list(map(totime,coinbase17['Timestamp'].values))
coinbase17.index

Prices from last year plotted together with volume (in units of BTC). Weighted price just means volume (in US$) divided by volume (in BTC). `gca` stands for "get the current axes instance". 

In [94]:
dates=[dt.datetime.fromtimestamp(ts) for ts in coinbase17['Timestamp'].values]
datenums=md.date2num(dates)
plt.subplots_adjust(bottom=0.2)
plt.xticks( rotation=25 )
ax=plt.gca()
xfmt = md.DateFormatter('%m-%d')
ax.xaxis.set_major_formatter(xfmt)
plt.plot(dates,coinbase17['Weighted_Price'].values)
plt.plot(dates,coinbase17['Volume_(BTC)'].values)
plt.show()

Let us also look at the currecy volume in US$.

In [95]:
plt.subplots_adjust(bottom=0.2)
plt.xticks( rotation=25 )
ax=plt.gca()
xfmt = md.DateFormatter('%m-%d')
ax.xaxis.set_major_formatter(xfmt)
plt.plot(dates,coinbase17['Volume_(Currency)'].values/1000)
plt.show()

If you perform FFT on real input you get $\frac{n}{2}+1$ outputs for $n$ input if $n$ is even. The one is for the constant term and the remaining $\frac{n}{2}$ contains the other nontrivial frequencies. But among these, the last one has a real part of zero.

In [96]:
rdft=np.fft.rfft(coinbase17['Weighted_Price'])
print(coinbase17['Weighted_Price'].shape)
print(rdft.shape)

In [97]:
525600/2+1

If you just do FFT, it will return $n$ outputs for $n$ inputs, again assuming $n$ is even. First one is again the constant term. Now you have 525599 numbers. 525598/2 of them is a conjugate of another 525598/2. There is another one that is a real number.

In [98]:
dft=np.fft.fft(coinbase17['Weighted_Price'])
print(dft.shape)

$A_k=\sum_{m=0}^{n-1} a_m \exp\left\lbrace-2\pi i \frac{mk}{n}\right\rbrace$ for $k$ from 0 to $n-1$.  Lets see this in more detail. 
\begin{align*}
A_k&=\sum_{m=0}^{n-1} a_m \exp\left\lbrace-2\pi i \frac{mk}{n}\right\rbrace\\
&=\sum_{m=0}^{n-1} a_m  \cos\left(-2\pi\frac{mk}{n}\right)+i\sum_{m=0}^{n-1} a_m  \sin\left(-2\pi\frac{mk}{n}\right)
\end{align*}
Lets see what happens if we replace $\frac{k}{n}$, the frequency, by $-\frac{k}{n}$.
\begin{align*}
&\sum_{m=0}^{n-1} a_m  \cos\left(2\pi\frac{mk}{n}\right)+i\sum_{m=0}^{n-1} a_m  \sin\left(2\pi\frac{mk}{n}\right)\\
&=\sum_{m=0}^{n-1} a_m  \cos\left(-2\pi\frac{mk}{n}\right)-i\sum_{m=0}^{n-1} a_m  \sin\left(-2\pi\frac{mk}{n}\right)
\end{align*}
Which is just the complex conjugate of the original. 


In [99]:
freqs=np.fft.fftfreq(525600)
freqs

In [100]:
freqs2=freqs[1:]
print(freqs2.shape)
print(freqs2)

In [101]:
(525599+1)/2

In [102]:
freqs2[262799]

In [103]:
freqs2[262797:262802]

In the middle it is $-\frac{1}{2}$. Lets see what is done for the real case. 

In [104]:
rfreqs=np.fft.rfftfreq(525600)
rfreqs

So thats a little bit confusing. It uses \frac{1}{2} instead. But that is not an issue because $\sin(n\frac{\pi}{2})=0$. And $\cos$ is symmetric about 0. Lets see how the values match.

In [105]:
dft[0]

In [106]:
print(dft[1:262800].shape)
dft[1:262800]

In [107]:
print(dft[:262800:-1].shape)
dft[:262800:-1]

In [108]:
np.conj(dft[:262800:-1])

In [109]:
rdft.shape

In [110]:
rdft[0]

In [111]:
dft[0]

In [112]:
rdft[1:262800]

In [113]:
rdft[262800]

In [114]:
dft[262800]

So basically, you can just always use `rfft`. And if you want the $\cos$ transform, take the real part. If you want the $\sin$ transform, take the imaginary part. 